## Expenses

This notebook tries to visualize your annual expenses.

### How to use

 - `YEAR` is the year.
 - `EXPENSE_FILEPATH` is the path to a TOML file containing all your expenses
 - `FONT_FAMILY` for font type
 - `FONT_SIZE` for font size

### Toml format
Expenses should be listed as such:
```toml
[[expense]]
date = 2021-01-10T00:00:00-05:00
description = "Toilet Seat"
price = 28.23
source = "Home Depot"
categories = ["Home Appliance"]

[[expense]]
date = 2021-11-03T12:32:00-05:00
description = "Bubble Tea with friend"
price = 8.69
source = "Blossoming Juice"
categories = ["Bubble Tea"]
```


In [ ]:
## configurations

YEAR = 2022

TRANSACTION_FILENAME_FORMAT = "TOML"

TRANSACTION_DIRECTORY = f"{YEAR}"
TRANSACTION_FILENAME = "expenses.toml"
TRANSACTION_FILEPATH = f"{TRANSACTION_DIRECTORY}/{TRANSACTION_FILENAME}"

FONT_FAMILY = "DejaVu Sans"
FONT_SIZE = 22

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum
import toml

## classes

class Transaction:
    def from_dict(d):
        t = Transaction()
        if 'date' in d:
            t.set_date(d['date'])
        if 'description' in d:
            t.set_description(d['description'])
        if 'value' in d:
            t.set_value(d['value'])
        if 'currency' in d:
            t.set_currency(d['currency'])
        if 'source' in d:
            t.set_source(d['source'])
        if 'categories' in d:
            t.set_categories(d['categories'])
        return t
    
    def set_date(self, date):
        self.date = date
        return self

    def set_description(self, description):
        self.description = description
        return self

    def set_value(self, value):
        self.value = value
        return self

    def set_currency(self, currency):
        self.currency = currency
        return self

    def set_source(self, source):
        self.source = source
        return self

    def set_categories(self, categories):
        self.categories = categories
        return self
    
    # getters
    def get_date(self):
        return self.date

    def get_description(self):
        return self.description

    def get_value(self):
        return self.value

    def get_currency(self):
        return self.currency

    def get_source(self):
        return self.source

    def get_categories(self):
        return self.categories

In [ ]:
## functions

def get_data(filename):
    monthly_transactions = [[] for i in range(12)]
    data = toml.load(filename)
    
    exchange_rate = data['exchange_rate']
    
    items = data['item']
    for item in items:
        tx = Transaction.from_dict(item)
        monthly_transactions[tx.get_date().month - 1].append(tx)

    return {
        "exchange_rate": exchange_rate,
        "balance_sheet": items,
        "monthly_transactions": monthly_transactions
    }


In [ ]:
# load

data = get_data(TRANSACTION_FILEPATH)

In [ ]:
# set font

font = {'family' : FONT_FAMILY,
        'weight' : 'bold',
        'size'   : FONT_SIZE}
plt.rc('font', **font)

## Monthly stats

In [ ]:
months = np.arange(1, 13)

def get_monthly_sums(data):
    exchange_rate = data['exchange_rate']
    monthly_transactions = data["monthly_transactions"]

    monthly_totals = []
    for month in monthly_transactions:
        monthly_total = 0
        for tx in month:
            tx_currency = tx.get_currency()
            if tx_currency != 'CAD' and tx_currency in exchange_rate:
                monthly_total += tx.get_value() * -1 / exchange_rate[tx_currency]
            else:
                monthly_total += tx.get_value() * -1
        monthly_totals.append(monthly_total)
    return monthly_totals

monthly_transactions = get_monthly_sums(data)
monthly_aggregate_transactions = np.cumsum(monthly_transactions)


# Graphing

print("Month\tTransactions")
for i, m in enumerate(monthly_transactions):
    print("{:02d}\t$ {:.02f}".format(i + 1, m))
print("Total:\t$ {:.02f}".format(sum(monthly_transactions)))

fig = plt.figure(figsize=(20, 10))
ax = fig.add_axes([0, 0, 1, 1])

ax.set_title("{} Monthly Transactions".format(YEAR))
ax.set_ylabel("CAD $", fontsize=FONT_SIZE)
ax.set_xlabel("Month", fontsize=FONT_SIZE)
ax.set_xticks(months)

# plt.plot(months, monthly_expenses)
ax.fill_between(months, 0, monthly_aggregate_transactions, color="#A89B8C")
ax.fill_between(months, 0, monthly_transactions, color="#F0DFAD")

for (i, v) in enumerate(monthly_aggregate_transactions):
    ax.text(i + 0.7, v + 2,
            "{:.02f}".format(v),
            color='#1E2749',
            fontweight='normal',
            fontsize=FONT_SIZE)

for (i, v) in enumerate(monthly_transactions):
    ax.text(i + 0.7, v + 2,
            "{:.02f}".format(v),
            color='#1E2749',
            fontweight='normal',
            fontsize=FONT_SIZE)

## Category Stats

In [ ]:
import itertools

CATEGORY_QUANTITY_KEY = 0
CATEGORY_PRICE_KEY = 1

FONT_SIZE = 24

all_categories = set()
all_category_values = {}

for monthly_txs in data["monthly_transactions"]:
    for tx in monthly_txs:
        tx_categories = tx.get_categories()
        all_categories.update(tx_categories)
        for category in tx_categories:
            if category not in all_category_values:
                all_category_values[category] = {CATEGORY_QUANTITY_KEY: 0, CATEGORY_PRICE_KEY: 0.0}

            all_category_values[category][CATEGORY_QUANTITY_KEY] += 1
            all_category_values[category][CATEGORY_PRICE_KEY] += tx.get_value() * -1

sorted_by_price = [(category, (v[CATEGORY_PRICE_KEY], v[CATEGORY_QUANTITY_KEY])) for (category, v) in all_category_values.items()]
sorted_by_price.sort(key=lambda x: x[CATEGORY_PRICE_KEY])
for (k, v) in sorted_by_price:
    print(k.ljust(20), '\t{}'.format(v[CATEGORY_PRICE_KEY]), "\t{:.02f}".format(v[CATEGORY_QUANTITY_KEY]))

monthly_transactions = [sum(tx.get_value() * -1 for tx in m) for m in data["monthly_transactions"]]

print("Total:".ljust(20), '\t$\t{:.02f}'.format(sum(monthly_transactions)))

sorted_by_price = [(category, v[CATEGORY_PRICE_KEY]) for (category, v) in all_category_values.items()]
sorted_by_price.sort(key=lambda x: x[1])

prices = [price for (category, price) in sorted_by_price]
xlabels = [category for (category, price) in sorted_by_price]

fig = plt.figure(figsize=(20, 10))
ax = fig.add_axes([0, 0, 1, 1])

ax.set_title("{} Category Expenses".format(YEAR))
ax.set_xlabel("CAD $")
ax.set_ylabel("Categories")
ax.set_yticks(np.arange(len(xlabels)))
ax.set_yticklabels(xlabels, fontsize=FONT_SIZE)

xs = [i for i in range(0, len(all_categories))]

ax.barh(xs, prices, color="#A89B8C")